Things to do:
*     Add NN, Knn, CatBoost, and other models to the blend.
*     Better preprocessing of the data - remove outliers, better take care of categorical features, remove NAs, feature  engineering, scaling values differently
*     Implement a linear regression model to stack oof values
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import random
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import gc
from math import sqrt, log
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load in training and test set
train=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
#Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price.
train["SalePrice"]=np.log1p(train["SalePrice"])

In [ ]:
target=train["SalePrice"]

In [ ]:
#Outliers that I found from another notebook
outliers = [30, 88, 462, 631, 1322]
train = train.drop(train.index[outliers])
train.reset_index(inplace=True)

di={"1Story":1,"2Story":2,"1.5Fin":1.5,"1.5Unf":1.5,"2.5Unf":2.5,"2.5Fin":2.5}
data["Story"]=data['HouseStyle'].map(di)

data["Story2"]=0
data.loc[data["MSSubClass"].isin([20,30,40,120]),"Story2"]=1
data.loc[data["MSSubClass"].isin([45,50,150]),"Story2"]=1.5
data.loc[data["MSSubClass"].isin([60,70,160]),"Story2"]=2
data.loc[data["MSSubClass"].isin([75]),"Story2"]=2.5

In [ ]:
#feature engineering/transforming data into something that can be put into a model using labelencoding or keeping
#the feature as a category
test["SalePrice"]=0
data=pd.concat([train,test])


data["LowDenseOrFV"]=0
data.loc[data["MSZoning"].isin(["FV","RL"]),"LowDenseOrFV"]=1

data["StreetToYard"]=data["LotFrontage"]/data["LotArea"]

data["gravel"]=0
data.loc[data["Street"]=="Grvl","gravel"]+=1
data.loc[data["Alley"]=="Grvl","gravel"]+=1


value_count=train["Neighborhood"].value_counts()/len(train)
value=value_count.index
for i,v in enumerate(value):
    data.loc[data['Neighborhood']==v,"NeighborhoodFreq"]=value_count[i]

frequencyEncode=["Electrical","Heating","Foundation","MasVnrType","Exterior2nd","Exterior1st","RoofStyle","HouseStyle","BldgType","Condition1","LotConfig","LotShape","MSZoning","MSSubClass"]
for feature in frequencyEncode:
    value_count=train[feature].value_counts()/len(train)
    value=value_count.index
    for i,v in enumerate(value):
        data.loc[data[feature]==v,feature+"Freq"]=value_count[v]
    
    
value_count=train["RoofMatl"].value_counts()/len(train)
value=value_count.index
for i,v in enumerate(value):
    data.loc[data['RoofMatl']==v,"RoofMatlFreq"]=value_count[i]

#di={"Norm":"Norm","Artery":"Artery","Feedr":"Feedr","RRAn":"RR","RRAe":"RR","RRNn":"RR","RRNe":"RR","PosA":"Pos","PosN":"Pos"}
#data['Condition1']=data['Condition1'].map(di)
#data['Condition2']=data['Condition2'].map(di)
di={"1Story":1,"2Story":2,"1.5Fin":1.5,"SLvl":1.2,"SFoyer":1.1,"1.5Unf":1.5,"2.5Unf":2.5,"2.5Fin":2.5}
data["story"]=data['HouseStyle'].map(di)
di={"1Story":0,"2Story":0,"1.5Fin":1,"SLvl":0,"SFoyer":0,"1.5Unf":1,"2.5Unf":1,"2.5Fin":0}
data["Unfinished"]=data['HouseStyle'].map(di)

data.loc[data["Exterior2nd"]=='Wd Shng',"Exterior2nd"]="Wd Sdng"
data.loc[data["Exterior2nd"]=='CmentBd',"Exterior2nd"]="CemntBd"
data["DifferentExterior"]=0
data.loc[data['Exterior1st']!=data['Exterior2nd'],"DifferentExterior"]=1
data["ExteriorComb"]=data['Exterior1st']+data['Exterior2nd']

import math
data['HasGarage']=data['GarageQual'].apply((lambda x: 1 if type(x)!=str else 0))
data['LotFrontNa']=data['LotFrontage'].apply((lambda x: 1 if math.isnan(x) else 0))
data['AlleyNa']=data['Alley'].apply((lambda x: 1 if type(x)!=str else 0))
data['VnrNa']=data['MasVnrType'].apply((lambda x: 1 if type(x)!=str else 0))
data['BsmtNa']=data['BsmtQual'].apply((lambda x: 1 if type(x)!=str else 0))
data['FireplaceNa']=data['FireplaceQu'].apply((lambda x: 1 if type(x)!=str else 0))
data['GarageNa']=data['GarageQual'].apply((lambda x: 1 if type(x)!=str else 0))
data['PoolNa']=data['PoolQC'].apply((lambda x: 1 if type(x)!=str else 0))
data['FenceNa']=data['Fence'].apply((lambda x: 1 if type(x)!=str else 0))
data['MiscNa']=data['MiscFeature'].apply((lambda x: 1 if type(x)!=str else 0))

#noneNa=["Alley","GarageType"]
#for feat in noneNa:
#    data[feat].fillna("None",inplace=True)
    
labelEncode=["MSSubClass","LotConfig","LandContour","MSZoning","RoofMatl","Neighborhood","Condition1","Condition2",
             "BldgType",'RoofStyle',"RoofMatl","Exterior1st","Exterior2nd","ExteriorComb","MasVnrType",
            "Foundation","Heating","GarageType","SaleType","SaleCondition","Electrical","HouseStyle",
            "Street","Alley","LandContour","LotConfig"]
#for feature in labelEncode:
#    le = LabelEncoder()
#    data[feature].fillna('NaN',inplace=True)
#    le.fit(data[feature])
#    data[feature]=le.transform(data[feature])
for c in labelEncode:
    data[c] = data[c].astype('category')


QualityScale={"Reg":3,"IR1":2,"IR2":1,"IR3":0}
data['LotShape']=data['LotShape'].map(QualityScale)

QualityScale={"Gtl":2,"Mod":1,"Sev":0}
data['LandSlope']=data['LandSlope'].map(QualityScale)

QualityScale={"Ex":4,"Gd":3,"TA":2,"Fa":1,"Po":0}
data['ExterQual']=data['ExterQual'].map(QualityScale)
data['ExterCond']=data['ExterCond'].map(QualityScale)

QualityScale={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}
data['BsmtQual']=data['BsmtQual'].map(QualityScale)
data['BsmtQual'].fillna(0,inplace=True)
data['BsmtCond']=data['BsmtCond'].map(QualityScale)
data['BsmtCond'].fillna(0,inplace=True)

QualityScale={"Gd":4,"Av":3,"Mn":2,"No":1}
data['BsmtExposure']=data['BsmtExposure'].map(QualityScale)
data['BsmtExposure'].fillna(0,inplace=True)

QualityScale={"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1}
data['BsmtFinType1']=data['BsmtFinType1'].map(QualityScale)
data['BsmtFinType1'].fillna(0,inplace=True)
data['BsmtFinType2']=data['BsmtFinType2'].map(QualityScale)
data['BsmtFinType2'].fillna(0,inplace=True)



QualityScale={"Ex":4,"Gd":3,"TA":2,"Fa":1,"Po":0}
data['HeatingQC']=data['HeatingQC'].map(QualityScale)

QualityScale={"Y":1,"N":0}
data['CentralAir']=data['CentralAir'].map(QualityScale)


QualityScale={"Ex":4,"Gd":3,"TA":2,"Fa":1,"Po":0}
data['KitchenQual']=data['KitchenQual'].map(QualityScale)

di={"Typ":1,"Min1":2,"Min2":3,"Mod":4,"Maj1":5,"Maj2":6,"Sev":7,"Sal":8}
data["Functional"]=data['Functional'].map(di)

#di={"1Story":1,"SLvl":2,"SFoyer":3,"1.5Fin":4,"1.5Unf":5,"2Story":6,"2.5Unf":7,"2.5Fin":8}
#data["HouseStyle"]=data['HouseStyle'].map(di)

QualityScale={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}
data['FireplaceQu']=data['FireplaceQu'].map(QualityScale)
data['FireplaceQu'].fillna(0,inplace=True)

QualityScale={"Fin":3,"RFn":2,"Unf":1}
data['GarageFinish']=data['GarageFinish'].map(QualityScale)
data['GarageFinish'].fillna(0,inplace=True)

QualityScale={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}
data['GarageQual']=data['GarageQual'].map(QualityScale)
data['GarageQual'].fillna(0,inplace=True)


QualityScale={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}
data['GarageCond']=data['GarageCond'].map(QualityScale)
data['GarageCond'].fillna(0,inplace=True)

QualityScale={"Y":2,"P":1,"N":0}
data['PavedDrive']=data['PavedDrive'].map(QualityScale)

QualityScale={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1}
data['PoolQC']=data['PoolQC'].map(QualityScale)
data['PoolQC'].fillna(0,inplace=True)

QualityScale={"GdPrv":4,"MnPrv":3,"GdWo":2,"MnWw":1}
data['Fence']=data['Fence'].map(QualityScale)
data['Fence'].fillna(0,inplace=True)


#oneHotEncode=["Exterior1st","SaleCondition"]
#data=pd.get_dummies(data,prefix=oneHotEncode, columns = oneHotEncode, drop_first=False)
#oneHotFeatures=list(data.columns[88:])
data["TimeElapsed"]=data['YearRemodAdd']-data["YearBuilt"]

data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

features=['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'YrSold', 'StreetToYard',
       'NeighborhoodFreq', 'ElectricalFreq',
       'TotalSF','LandSlope',"LotShape"]

from scipy.stats import norm, skew 
numeric_feats = features
skewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.Series(skewed_feats)
skewness.head(10)

skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    data[feat] = boxcox1p(data[feat], lam)

In [ ]:
#Define code for LIGHTGBM MODEL also define which features will be used. 
features=["Fence","HalfBath","FullBath","Heating","RoofMatl","LandSlope","LotConfig","LandContour","Alley","Street","YearBuilt","OverallQual","OverallCond","MSSubClass",'LowDenseOrFV',"LotFrontage",
          "LotArea","StreetToYard","gravel","LotShape","NeighborhoodFreq","Neighborhood","Condition1","Condition2",
          "BldgType","HouseStyle","Unfinished","YearRemodAdd",'RoofStyle',"Exterior1st","Exterior2nd","DifferentExterior",
          "MasVnrType","MasVnrArea","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1",
          'BsmtFinSF1',"BsmtFinType2","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF",'HeatingQC',"CentralAir","Electrical",
         "1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath","BsmtHalfBath",
         'BedroomAbvGr', 'KitchenAbvGr',"KitchenQual","TotRmsAbvGrd","Functional","Fireplaces","FireplaceQu","GarageType","GarageYrBlt",
         "GarageFinish",'GarageCars',"GarageArea","GarageQual","GarageCond","PavedDrive",
         "WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","YrSold","SaleType","SaleCondition","PoolQC","TotalSF","ElectricalFreq"]
#         'ElectricalFreq', 'HeatingFreq', 'FoundationFreq', 'MasVnrTypeFreq', 'Exterior2ndFreq', 'Exterior1stFreq', 'RoofStyleFreq', 
#          'HouseStyleFreq', 'BldgTypeFreq', 'Condition1Freq', 'LotConfigFreq', 'LotShapeFreq', 'MSZoningFreq', 'MSSubClassFreq']#+oneHotFeatures
#"HasGarage","LotFrontNa","AlleyNa","VnrNa","BsmtNa","FireplaceNa",'GarageNa',"PoolNa","FenceNa","MiscNa"
#features=featuresOld[:]
#for feat in featuresOld:
#    if feat in oneHotEncode:
#        features.remove(feat)

import lightgbm as lgb
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=2):
    
    folds = KFold(n_splits=NFOLDS)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  
    
    tt_df = tt_df[['Id',target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y)  

        estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 1000,
        )   
        
        pp_p = estimator.predict(P)
        predictions += pp_p/NFOLDS
        
        oof_preds = estimator.predict(vl_x)
        oof[val_idx] = oof_preds

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF RMSE:', sqrt(metrics.mean_squared_error(y,oof)))
    #print('OOF RMSE:', sqrt(metrics.mean_squared_error(y.apply(lambda x: log(x)),np.log(oof))))
    if LOCAL_TEST:
        print('Holdout RMSE:', sqrt(metrics.mean_squared_error(tt_df[TARGET], tt_df['prediction'])))
    
    return (predictions,oof, estimator)

SEED = 42
seed_everything(SEED)
LOCAL_TEST = False
trainIndex=train.index
testIndex=list(range(len(trainIndex),len(data)))
lgb_params = {
                    'objective':'regression',
                    'boosting_type':'gbdt',
                    'metric':'mae',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    #'num_leaves': 5,
                    'num_leaves': 10,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':800,
                    'max_bin':200,
                    'verbose':-1,
                    'seed': SEED,
                    'early_stopping_rounds':500, 
                } 

In [ ]:
lgb_params['learning_rate'] = 0.01
lgb_params['n_estimators'] = 10000
lgb_params['early_stopping_rounds'] = 500
test_predictions_lgbm,oof_predictions_lgbm,clf = make_predictions(data.iloc[trainIndex,:], data.iloc[testIndex,:], features, "SalePrice", lgb_params, NFOLDS=4)

In [ ]:
features=["Fence","HalfBath","FullBath","Heating","RoofMatl","LandSlope","LotConfig","LandContour","Alley","Street","YearBuilt","OverallQual","OverallCond","MSSubClass",'LowDenseOrFV',"LotFrontage",
          "LotArea","StreetToYard","gravel","LotShape","NeighborhoodFreq","Neighborhood","Condition1","Condition2",
          "BldgType","HouseStyle","Unfinished","YearRemodAdd",'RoofStyle',"Exterior1st","Exterior2nd","DifferentExterior",
          "MasVnrType","MasVnrArea","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1",
          'BsmtFinSF1',"BsmtFinType2","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF",'HeatingQC',"CentralAir","Electrical",
         "1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath","BsmtHalfBath",
         'BedroomAbvGr', 'KitchenAbvGr',"KitchenQual","TotRmsAbvGrd","Functional","Fireplaces","FireplaceQu","GarageType","GarageYrBlt",
         "GarageFinish",'GarageCars',"GarageArea","GarageQual","GarageCond","PavedDrive",
         "WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","YrSold","SaleType","SaleCondition","PoolQC","TotalSF","ElectricalFreq"]

In [ ]:
#XGBoost can't use category data.  Have to one hot encode categories that were used in the lightgbm model
oneHotEncode=["Street","Alley",
              "MSSubClass","LotConfig","LandContour","LandSlope","MSZoning","RoofMatl","Neighborhood","Condition1","Condition2",
             "BldgType",'RoofStyle',"Exterior1st","Exterior2nd","ExteriorComb","MasVnrType",
            "Foundation","Heating","GarageType","SaleType","SaleCondition","Electrical","LotShape","HouseStyle"]
#newFeatures=["ExterQual",'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',"HeatingQC","CentralAir","KitchenQual","Functional","HouseStyle","FireplaceQu","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC"]
#newFeatures=['YrSold','Fence', 'MiscFeature', 'MoSold']
oneHotEncode=oneHotEncode#+newFeatures
start=len(data.columns)-len(oneHotEncode)
data=pd.get_dummies(data,prefix=oneHotEncode, columns = oneHotEncode, drop_first=True)
oneHotFeatures=list(data.columns[start:])
featuresOld=features[:]
for feat in featuresOld:
    if feat in oneHotEncode:
        features.remove(feat)
features=features+oneHotFeatures

In [ ]:
#define xgboost model
def make_predictions(tr_df, tt_df, features_columns, target, xgb_params, NFOLDS=2):
    
    folds = KFold(n_splits=NFOLDS)
    

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  
    d_test = xgb.DMatrix(P)
    
    tt_df = tt_df[['Id',target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]

        tr_data = xgb.DMatrix(tr_x, label=tr_y)
        vl_data = xgb.DMatrix(vl_x, label=vl_y)


        estimator = xgb.train(
            xgb_params,
            tr_data,
            10000,
            evals = [(tr_data,'train'),(vl_data,'eval')],
            early_stopping_rounds=500,
            maximize=False, 
            verbose_eval = 1000,
        )   
        
        pp_p = estimator.predict(d_test)
        predictions += pp_p/NFOLDS
        
        oof_preds = estimator.predict(vl_data)
        oof[val_idx] = oof_preds

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF RMSE:', sqrt(metrics.mean_squared_error(y,oof)))
    #print('OOF RMSE:', sqrt(metrics.mean_squared_error(y.apply(lambda x: log(x)),np.log(oof))))
    if LOCAL_TEST:
        print('Holdout RMSE:', sqrt(metrics.mean_squared_error(tt_df[TARGET], tt_df['prediction'])))
    
    return (predictions,oof, estimator)


xgb_params = {
        'objective':'reg:linear',
        'n_estimators': 10000,
        'booster':'gbtree',
        'max_depth':4,
        'eval_metric':'mae',
        'learning_rate':0.005, 
        'min_child_weight':2,
        'subsample':0.7,
        'colsample_bytree':0.8,
        'seed':45,
        'reg_alpha':1,
        'gamma':0,
        'nthread':-1

}

In [ ]:
#run xgboost
test_predictions_xgb,oof_predictions_xgb,clf = make_predictions(data.iloc[trainIndex,:], data.iloc[testIndex,:], features, "SalePrice", xgb_params, NFOLDS=4)

In [ ]:
#Get rid of na values, linear models can't use NA's
setMean=["LotFrontage",'StreetToYard','KitchenQual','Functional',"ElectricalFreq"]                                                     
for feat in setMean:
    data.loc[data[feat].isna(),feat]=np.mean(data[feat])

setZero=['MasVnrArea','BsmtFinSF1','BsmtUnfSF',"BsmtFinSF2",'TotalBsmtSF','BsmtFullBath','BsmtHalfBath',"GarageCars","GarageArea","TotalSF"]                                               
for feat in setZero:
    data.loc[data[feat].isna(),feat]=0

setMin=["GarageYrBlt"]                                                    
for feat in setMin:
    data.loc[data[feat].isna(),feat]=np.min(data[feat])


In [ ]:
def cv_rmse(model, X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y,
                                    scoring="neg_mean_squared_error",
                                    cv=10))
    return (rmse)

In [ ]:
#define ridge regression model
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge
def make_predictions(tr_df, tt_df, features_columns, target, xgb_params, NFOLDS=2):
    
    folds = KFold(n_splits=NFOLDS)
    

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  

    
    tt_df = tt_df[['Id',target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]

        tr_data = xgb.DMatrix(tr_x, label=tr_y)
        vl_data = xgb.DMatrix(vl_x, label=vl_y)


        clf = ridge = make_pipeline(RobustScaler(),
                      Ridge()) #make_pipeline(StandardScaler(), SVR(gamma='auto'))
        clf.fit(tr_x, tr_y)
        
        pp_p = clf.predict(P)
        predictions += pp_p/NFOLDS
        
        oof_preds = clf.predict(vl_x)
        oof[val_idx] = oof_preds


        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF RMSE:', sqrt(metrics.mean_squared_error(y,oof)))
    #print('OOF RMSE:', sqrt(metrics.mean_squared_error(y.apply(lambda x: log(x)),np.log(oof))))

    return (predictions,oof, clf)

In [ ]:
#run ridge regression
test_predictions_Ridge,oof_predictions_Ridge,clf = make_predictions(data.iloc[trainIndex,:], data.iloc[testIndex,:], features, "SalePrice", xgb_params, NFOLDS=4)

In [ ]:
#This would be a good spot for the NN
from keras.models import Sequential
from keras.layers import Dense

def make_predictions(tr_df, tt_df, features_columns, target, NFOLDS=2):
    
    folds = KFold(n_splits=NFOLDS)
    

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  
    
    
    tt_df = tt_df[['Id',target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]

        tr_data = xgb.DMatrix(tr_x, label=tr_y)
        vl_data = xgb.DMatrix(vl_x, label=vl_y)
        model = Sequential()
        model.add(Dense(20,kernel_initializer='normal', activation='relu'))
        model.add(Dense(25, kernel_initializer='normal', activation='relu'))
        model.add(Dense(10, kernel_initializer='normal', activation='relu'))
        model.add(Dense(10, kernel_initializer='normal', activation='relu'))
        model.add(Dense(5, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(tr_x, tr_y, epochs=100, validation_split=0.2, verbose=0)
        

        
        pp_p = model.predict(P)
        predictions += pp_p/NFOLDS
        
        oof_preds = model.predict(vl_x)
        oof[val_idx] = oof_preds


        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF RMSE:', sqrt(metrics.mean_squared_error(y,oof)))
    #print('OOF RMSE:', sqrt(metrics.mean_squared_error(y.apply(lambda x: log(x)),np.log(oof))))

    return (predictions,oof, model)

In [ ]:
#run neural network
test_predictions_NN,oof_predictions_NN,model = make_predictions(data.iloc[trainIndex,:], data.iloc[testIndex,:], features, "SalePrice", NFOLDS=4)

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=10))
score = cv_rmse(ridge,data.iloc[trainIndex,:][features],data.iloc[trainIndex,:]["SalePrice"])
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=10))
score = cv_rmse(ridge,data.iloc[trainIndex,:][features],data.iloc[trainIndex,:]["SalePrice"])
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
#blend values
blended_oof=oof_predictions_lgbm*.6+oof_predictions_xgb*.15+oof_predictions_Ridge*.15
blended_test=test_predictions_lgbm*.6+test_predictions_xgb*.15+test_predictions_Ridge*.15 + test_predictions

In [ ]:
print('OOF RMSE:', sqrt(metrics.mean_squared_error(target,oof_predictions_lgbm)))
print('OOF RMSE:', sqrt(metrics.mean_squared_error(target,oof_predictions_xgb)))
print('OOF RMSE:', sqrt(metrics.mean_squared_error(target,oof_predictions_Ridge)))
print('OOF RMSE:', sqrt(metrics.mean_squared_error(target,blended_oof)))

In [ ]:
#uncorrelates oof predictions means blends will be more powerful.
np.corrcoef(oof_predictions_svr,oof_predictions_lgbm)

In [ ]:
submission=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
submission["SalePrice"]=np.expm1(blended_test)

In [ ]:
submission.to_csv("submission.csv",index=False)